In [1]:
# class Synapse (copyright Neuronycs 2023)

from numpy import arange, array, ones, transpose

#=============================================================================================
# class Synapses
#=============================================================================================

class Synapses:
    """
    Synapses: class
        syn = Synapses(K,P,eta,theta,(plus,minus))   # full arg list
        syn = Synapses(K,P)       # eta=0.5, plus=minus=0.02
        syn = Synapses(g)         # P=one 

        P = syn.P                 # permanences

        W = syn.W()               # synaptic weight matrix
        V = syn.V(c)              # presynaptic signals
        E = syn.E(c)              # empowering matrix
        L = syn.L(c)              # learning mask
        D = syn.D(c)              # learning deltas
        s = syn.s(c)              # spike vector
        o = syn.one               # [1,1,...,1] matrix (1 x ns)

        syn.P = syn.sat(P)        # truncate P matrix to range [0,1]
    """
    def __init__(self,K,P=None,eta=0.5,theta=2,delta=(0.02,0.02)):
        self.K = array(K)         # always store as numpy array
        self.P = array(P)         # always store as numpy array
        self.eta = eta            # synaptic threshold
        self.theta = theta        # spiking threshold
        self.delta = delta        # learning delta (plus,minus)
    
    def W(self):                  # binary weights
        return (self.P >= self.eta)*1
    
    def V(self,c):                    # pre-synaptic signals
        kmax = len(c)
        V = 0*self.K
        for mu in range(0,self.K.shape[0]):
            for nu in range(0,self.K.shape[1]):
                k = self.K[mu,nu]
                V[mu,nu] = c[k] if k < kmax else 0;
        return V

    def E(self,c):                     # E = V(c) * W(P)
        return self.V(c) * self.W()    # empowerment matrix

    def L(self,c):
        one = ones(K.shape)
        return transpose(array([self.s(c)])) @ array([one[0]])

    def D(self,c):                     # D = (2*plus * V - minus) * L
        L = self.L(c);  V = self.V(c);  
        plus,minus = self.delta
        return (2*plus * V - minus) * L
        
    def s(self,c):                     # spike vector: s = (sum(E')>=theta)
        E = self.E(c)
        s = [(sum(E[mu]) >= self.theta)
             for mu in range(0,E.shape[0])]
        return 1*array(s)

    def v(self,c):                     # group output
        return array([c[k] if k < len(c) else 0
                      for k in self.K])
        
    def sat(X):  # truncates every matrix element of X to range 0.0 ... 1.0
        def lt1(X): return 1 + (X-1<=0)*(X-1)
        def gt0(X): return (X>=0)*X
        return lt1(gt0(X))



In [2]:
# synaptic bank

from carabao.cell import toy
k,g,K,P,c = toy('cell')
syn = Synapses(K,P)
c = [1,1,1,1,1,1,1,1,1,1]; print("c:",repr(c)) 
P=syn.P;    print("syn.P   ",repr(P))
W=syn.W();  print("syn.W() ",repr(W))
V=syn.V(c); print("syn.V(c)",repr(V)) 
E=syn.E(c); print("syn.E(c)",repr(E))
L=syn.L(c); print("syn.L(c)",repr(L))
D=syn.D(c); print("syn.D(c)",repr(D))
s=syn.s(c); print("syn.s(c)",repr(s))

c: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
syn.P    array([[0.12, 0.32, 0.54, 0.77, 0.  ],
       [0.  , 0.61, 0.45, 0.  , 0.8 ]])
syn.W()  array([[0, 0, 1, 1, 0],
       [0, 1, 0, 0, 1]])
syn.V(c) array([[1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1]])
syn.E(c) array([[0, 0, 1, 1, 0],
       [0, 1, 0, 0, 1]])
syn.L(c) array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]])
syn.D(c) array([[0.02, 0.02, 0.02, 0.02, 0.02],
       [0.02, 0.02, 0.02, 0.02, 0.02]])
syn.s(c) array([1, 1])


In [3]:
# synaptic field

from carabao.cell import toy
k,g,K,P,c = toy('cell')
syn = Synapses(g)
c = [0,1,0,1,0,0,0,0,0,0]; print("c:",repr(c)) 
v=syn.v(c); print("syn.v(c)",repr(v)) 


c: [0, 1, 0, 1, 0, 0, 0, 0, 0, 0]
syn.v(c) array([0, 1, 0, 1])
